In [1]:
from eaaw_graphlime_utils import *
import gc

import seaborn as sns
import seaborn.objects as so


from   pcgrad.pcgrad import PCGrad # from the following: https://github.com/WeiChengTseng/Pytorch-PCGrad. Renamed to 'pcgrad' and moved to site-packages folder.

import pandas as pd

# torch.manual_seed(2)

ScalarFunction


In [14]:
dataset_name='Flickr'
if dataset_attributes[dataset_name]['single_or_multi_graph']=='single':
    dataset = prep_data(dataset_name=dataset_name, location='default', batch_size='default', transform_list='default',
                        train_val_test_split=[0.9,0.05,0.05])
    graph_to_watermark = data = dataset[0]
elif dataset_attributes[dataset_name]['single_or_multi_graph']=='multi':
    [train_dataset, val_dataset, test_dataset], [train_loader, val_loader, test_loader] = prep_data(dataset_name=dataset_name, location='default', 
                                                                                                    batch_size='default', transform_list='default',
                                                                                                                            train_val_test_split=[0.9,0.05,0.05])
    graph_to_watermark = train_dataset[0]

class: <class 'torch_geometric.datasets.flickr.Flickr'>
Transorms used when loading Flickr: ['NormalizeFeatures()', 'ChooseLargestMaskForTrain()']
train_mask: 44625
test_mask: 22313
val_mask: 22312


In [18]:
#optimization_kwargs, node_classifier_kwargs, watermark_kwargs, subgraph_kwargs, augment_kwargs, watermark_loss_kwargs, regression_kwargs = get_presets(dataset, 'default')
get_presets(dataset,dataset_name)

compare_unimportant_against_random=False

config.node_classifier_kwargs['dropout']=0.1
config.node_classifier_kwargs['dropout_subgraphs']=0
config.watermark_kwargs['unimportant_selection_kwargs']['clf_only_epochs'] = 20
config.optimization_kwargs['lr']=0.001
config.optimization_kwargs['epochs']=70
# config.optimization_kwargs['coefWmk']=300
config.optimization_kwargs['perturb_x']=False
config.optimization_kwargs['perturb_lr']=1e5

config.optimization_kwargs['coefWmk_kwargs']= {
                                                'coefWmk':50,
                                                'schedule_coef_wmk': False,
                                                'min_coefWmk_scheduled': 100,
                                                'reach_max_coef_wmk_by_epoch':70,
                                                }

config.optimization_kwargs['use_pcgrad']=True
config.optimization_kwargs['use_sam']=False
config.optimization_kwargs['sam_momentum']=0.5
config.optimization_kwargs['sam_rho']=1e-5

config.optimization_kwargs['penalize_similar_subgraphs']=False
config.optimization_kwargs['p_swap']=0.5
config.optimization_kwargs['shifted_subgraph_loss_coef']=1e-1


config.optimization_kwargs['use_gradnorm']=False


config.augment_kwargs['p']=0.3
config.augment_kwargs['nodeDrop']['use']=True
config.augment_kwargs['nodeDrop']['p']=0.1
config.augment_kwargs['nodeMixUp']['use']=True
config.augment_kwargs['nodeMixUp']['lambda']=5
config.augment_kwargs['nodeFeatMask']['use']=False
config.augment_kwargs['edgeDrop']['use']=True
config.augment_kwargs['edgeDrop']['p']=0.1

config.augment_kwargs['separate_trainset_from_subgraphs'] = True
config.augment_kwargs['ignore_subgraphs'] = True
config.watermark_kwargs['percent_of_features_to_watermark']=3
config.watermark_kwargs['watermark_type']='most_represented'
config.subgraph_kwargs['numSubgraphs']=7
config.subgraph_kwargs['fraction']=0.01
config.subgraph_kwargs['method']='random'
config.optimization_kwargs['sacrifice_kwargs']['method']='subgraph_node_indices'
config.optimization_kwargs['sacrifice_kwargs']['percentage']=1
config.optimization_kwargs['separate_forward_passes_per_subgraph']=False
config.optimization_kwargs['clf_only']=False
config.watermark_loss_kwargs['epsilon']=1e-1

config.regression_kwargs['lambda']=1e-3

validate_kwargs()


In [19]:
merged_dict = merge_kwargs_dicts()#config.node_classifier_kwargs, config.optimization_kwargs, config.watermark_kwargs, config.subgraph_kwargs, config.regression_kwargs, config.watermark_loss_kwargs, config.augment_kwargs)
merged_dict_keys = list(merged_dict.keys()) + ['Train Acc','Val Acc','Match Rates','Final Betas','watermark']
all_dfs = pd.DataFrame(dict(zip(merged_dict_keys,[]*len(merged_dict_keys))))
scale_beta_method=None
debug_multiple_subgraphs=False



In [20]:
config.optimization_kwargs['separate_forward_passes_per_subgraph']=True
# for type_ in ['unimportant','most_represented']:
type_ = 'most_represented'
config.watermark_kwargs['watermark_type'] = type_
if type_ == 'unimportant':
    config.optimization_kwargs['coefWmk_kwargs']['coefWmk'] = 50
    config.optimization_kwargs['epochs'] = 70
else:
    config.optimization_kwargs['coefWmk_kwargs']['coefWmk'] = 20
    config.optimization_kwargs['epochs'] = 90
for perc in [3,10,50]:
    config.watermark_kwargs['percent_of_features_to_watermark']=perc
    for num_sub in [2,5,10]:
        config.subgraph_kwargs['numSubgraphs']=num_sub
        title_ = f'Separate Forward Passes -- {type_} feature indices'
        title = f'{title_}.\n{config.subgraph_kwargs['numSubgraphs']} subgraphs.\nWatermarking {config.watermark_kwargs['percent_of_features_to_watermark']}% of node features'
        Trainer_ = Trainer(data, dataset_name)
        node_classifier, history, subgraph_dict, \
        all_feature_importances, all_watermark_indices, probas = Trainer_.train(debug_multiple_subgraphs=False, save=True, print_every=1)

        primary_loss_curve, watermark_loss_curve, final_betas, watermarks, \
        percent_matches, percent_match_mean, percent_match_std, \
            primary_acc_curve, watermark_acc_curve, train_acc, val_acc = get_performance_trends(history, subgraph_dict)
        final_plot(history, title, percent_matches, primary_loss_curve, watermark_loss_curve, train_acc)  


lambda: 5
enumerate_over_me: range(0, 2)
Before generate_subgraph: numHops = 1
selected_nodes: tensor([52043, 80982, 81279, 70818, 18272, 39812, 71085,  8955, 29682, 64421,
         9781, 78657, 55580, 24486, 51615, 29299, 36853, 70046, 69811,  2303,
        73512, 33737, 51021, 15574, 62714, 66927, 20956, 86705, 73681, 74292,
        19697, 54290, 64002, 47058, 28498, 75353, 71068, 31883, 44830, 27529,
        16994, 86528, 28127, 74993, 38129,  5175, 79800, 62001, 29788, 61327,
        41870, 25925, 52268, 57187, 12641,  9666, 37265, 69493, 72019, 16774,
         1944,  7908,  9449,  3685, 13293, 52316, 22058, 32318, 86067, 66819,
        63850, 71080, 26735, 24753, 17203, 36698, 55212, 24710, 74006, 76124,
        22756, 45164, 13493, 84571, 30563, 53242, 50712, 65328,  8664, 79655,
        15298, 13132, 83589,  2032, 55638, 61810, 84221, 55842, 29811, 14492,
        31179, 45758, 82179, 64560, 85563, 60739, 51975, 27937, 54351, 69499,
        73177, 78108, 43027,  6702, 18250, 6920

  0%|          | 0/90 [00:00<?, ?it/s]

len sorted_indices: 500
len_watermark: 15
most_represented_indices: tensor([ 62, 392, 425,  36, 302, 316,  40, 203, 482, 255, 156, 283, 442, 135,
        306])
watermarks: [tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,

OSError: [Errno 28] No space left on device

In [21]:
len(subgraph_dict)

10

In [ ]:
Trainer_.node_classifier.eval()
Trainer_.optimizer.zero_grad()
log_logits          = Trainer_.forward(data.x, data.edge_index, dropout=config.node_classifier_kwargs['dropout'])
acc_trn = accuracy(log_logits[Trainer_.train_mask], data.y[Trainer_.train_mask],verbose=False)
acc_val = accuracy(log_logits[Trainer_.val_mask],   data.y[Trainer_.val_mask],verbose=False)
print(acc_trn, acc_val)
Trainer_.node_classifier.eval()
Trainer_.optimizer.zero_grad()
log_logits          = Trainer_.forward(Trainer_.x, Trainer_.edge_index, dropout=config.node_classifier_kwargs['dropout'])
acc_trn = accuracy(log_logits[Trainer_.train_mask], Trainer_.y[Trainer_.train_mask],verbose=False)
acc_val = accuracy(log_logits[Trainer_.val_mask],   Trainer_.y[Trainer_.val_mask],verbose=False)
print(acc_trn, acc_val)

tensor(0.7551, dtype=torch.float64) tensor(0.7802, dtype=torch.float64)
tensor(0.8130, dtype=torch.float64) tensor(0.8195, dtype=torch.float64)


In [ ]:
def shift_subgraph(self, p_to_swap, subgraph_node_indices):
    num_to_swap = int(p_to_swap*len(subgraph_node_indices))
    random_indices = torch.randperm(len(subgraph_node_indices))
    subgraph_node_indices = subgraph_node_indices[random_indices[:len(subgraph_node_indices)-num_to_swap]]
    filtered_tensor = Trainer_.train_nodes_to_consider[~Trainer_.train_nodes_to_consider.unsqueeze(1).eq(subgraph_node_indices).any(dim=1)]
    random_index = torch.randint(0, filtered_tensor.size(0), (num_to_swap,))
    random_element = filtered_tensor[random_index]
    subgraph_node_indices = torch.concatenate([subgraph_node_indices, random_element])

    sub_edge_index, _ = subgraph(subgraph_node_indices, self.data.edge_index, relabel_nodes=True, num_nodes=self.data.num_nodes)
    shifted_subgraph = Data(
        x          = self.data.x[subgraph_node_indices]          if self.data.x is not None else None,
        edge_index = sub_edge_index,
        y          = self.data.y[subgraph_node_indices]          if self.data.y is not None else None,
        train_mask = self.data.train_mask[subgraph_node_indices] if self.data.train_mask is not None else None,
        test_mask  = self.data.test_mask[subgraph_node_indices]  if self.data.test_mask is not None else None,
        val_mask   = self.data.val_mask[subgraph_node_indices]   if self.data.val_mask is not None else None)
    return shifted_subgraph






In [ ]:
Trainer_.percent_matches

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]

In [ ]:
Trainer_.node_classifier.eval()
Trainer_.optimizer.zero_grad()


log_logits   = Trainer_.forward(data.x, data.edge_index, dropout=0)
print('Full prediction accuracies:')
for i, sig in enumerate(Trainer_.subgraph_dict.keys()):
    node_indices = Trainer_.subgraph_dict[sig]['nodeIndices']
    these_out = log_logits[node_indices]
    these_y = data.y[node_indices]
    acc = accuracy(these_out, these_y, verbose=False)
    print(acc)
for p_to_swap in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]:
    matches = []
    sign_betas = []
    for i, sig in enumerate(Trainer_.subgraph_dict.keys()):
        node_indices = Trainer_.subgraph_dict[sig]['nodeIndices']
        num_to_swap = int(p_to_swap*len(node_indices))

        # print('node indices start:', node_indices)
        random_indices = torch.randperm(len(node_indices))
        node_indices = node_indices[random_indices[:len(node_indices)-num_to_swap]]
        filtered_tensor = Trainer_.train_nodes_to_consider[~Trainer_.train_nodes_to_consider.unsqueeze(1).eq(node_indices).any(dim=1)]
        random_index = torch.randint(0, filtered_tensor.size(0), (num_to_swap,))
        random_element = filtered_tensor[random_index]
        node_indices = torch.concatenate([node_indices, random_element])
        # print('node indices end  :', node_indices)

        sub_edge_index, _ = subgraph(node_indices, data.edge_index, relabel_nodes=True, num_nodes=data.num_nodes)
        subgraph_ = Data(
            x=data.x[node_indices] if data.x is not None else None,
            edge_index=sub_edge_index,
            y=data.y[node_indices] if data.y is not None else None,
            train_mask=data.train_mask[node_indices] if data.train_mask is not None else None,
            test_mask=data.test_mask[node_indices] if data.test_mask is not None else None,
            val_mask=data.val_mask[node_indices] if data.val_mask is not None else None)
    

        x_sub, edge_index_sub = subgraph_.x, subgraph_.edge_index
        these_log_logits   = Trainer_.forward(x_sub, edge_index_sub, dropout=config.node_classifier_kwargs['dropout'])
        these_probas = these_log_logits.clone().exp()
        y_sub = these_probas

        watermark_loss_kwargs = config.watermark_loss_kwargs
        regression_kwargs = config.regression_kwargs
        this_watermark = subgraph_dict[sig]['watermark']#[subgraph_dict[sig][k] for k in ['watermark','subgraph','nodeIndices']]


        ''' epoch condtion: epoch==epoch-1'''
        omit_indices,not_omit_indices = get_omit_indices(x_sub, this_watermark,ignore_zeros_from_subgraphs=False) #indices where watermark is 0
        this_raw_beta = solve_regression(x_sub, y_sub, regression_kwargs['lambda'])
        beta  = process_beta(this_raw_beta, watermark_loss_kwargs['alpha'], omit_indices, watermark_loss_kwargs['scale_beta_method'])
        these_watermark_indices = Trainer_.all_watermark_indices[i]
        #print('watermark:',this_watermark[these_watermark_indices])
        #print('beta:     ',torch.sign(beta[these_watermark_indices]))
        match = torch.sum(this_watermark[these_watermark_indices]==torch.sign(beta[these_watermark_indices]))/len(these_watermark_indices)
        sign_betas.append(torch.sign(beta[these_watermark_indices]))
        #match_str = str(np.round(100*match.item(),3)) + '%'
        matches.append(100*match.item())
    print(f'Replacing {int(100*p_to_swap)}% watermark subgraph indices:')
    print('watermark match rates:',np.mean(matches))
    bs = torch.vstack(sign_betas)
    match_counts= count_matches(bs)
    print('count of shared sign(beta) values across subgraphs:',match_counts)


Full prediction accuracies:
tensor(0.8491, dtype=torch.float64)
tensor(0.7925, dtype=torch.float64)
tensor(0.7358, dtype=torch.float64)
tensor(0.7547, dtype=torch.float64)
tensor(0.6604, dtype=torch.float64)
tensor(0.7547, dtype=torch.float64)
tensor(0.7547, dtype=torch.float64)


AttributeError: 'Trainer' object has no attribute 'all_watermark_indices'

In [ ]:
len(these_watermark_indices)

23

In [ ]:
tensor = Trainer_.train_nodes_to_consider

# Separate list of elements to exclude
exclude_tensor = node_indices

# Convert exclude_list to a tensor for comparison
# exclude_tensor = torch.tensor(exclude_list)

# Filter the original tensor to exclude elements in the exclude list
filtered_tensor = tensor[~tensor.unsqueeze(1).eq(exclude_tensor).any(dim=1)]
random_index = torch.randint(0, filtered_tensor.size(0), (5,))
random_element = filtered_tensor[random_index]
node_indices = torch.concatenate([node_indices, random_element])

In [ ]:
Trainer_.x[torch.tensor(list(set(np.concatenate(Trainer_.all_watermark_indices))))]

tensor([[ 1.,  1.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [ 0., 10.,  0.,  ..., 10.,  0., 10.],
        ...,
        [10.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0., -9.,  0.,  ...,  0., -9.,  0.]])

In [ ]:
config.node_classifier_kwargs.keys()

dict_keys(['arch', 'activation', 'nLayers', 'hDim', 'dropout', 'dropout_subgraphs', 'skip_connections', 'heads_1', 'heads_2', 'inDim', 'outDim'])

In [ ]:

''' (individualize boolean, multisubgraph method, index selection method) '''
selection_kwargss = [#(False, None, 'random'), # random, not individualized
                    (False,'average','unimportant'), # average, not individualized
                    #(False,'concat','unimportant'), # concat, not inividualized
                    # (True,None,'unimportant') # unimportant, individualized
                    ]

''' (method, regenerate_boolean) '''
subgraph_methods = [('random',False)]

''' [subset, percentage] '''
sacrifice_kwargss = [['train_node_indices',1],['train_node_indices',0.75],['subgraph_node_indices',1],[None,None]]

''' (method, L2_lambda (if applicable, else None) '''
regularization_kwargs = [(None,None),('L2',0.01),('beta_var',None)]


variables = {'augment': [{'separate_trainset_from_subgraphs': True,
                          'ignore_subgraphs': True,
                          'nodeDrop': False,
                          'nodeMixUp': False,
                          'nodeFeatMask': False,
                          'edgeDrop': False},
                          {'separate_trainset_from_subgraphs': True,
                          'ignore_subgraphs': True,
                          'nodeDrop': True,
                          'nodeMixUp': True,
                          'nodeFeatMask': True,
                          'edgeDrop': True},],
             'sacrifice_kwargs':  sacrifice_kwargss,
             'beta_selection': selection_kwargss,
             'use_PCgrad': [True,False],
             'subgraph_method': subgraph_methods,
             'reg': regularization_kwargs,
             'perc': [3],
             'clf_epochs': [10,20],
             'coef_wmk': [200,600],
             'frac': [0.02],
             'num_subgraphs': [2,7],
             'balance_beta_weights': [True,False],
             'ignore_subgraph_neighbors': [True, False]}




# config.optimization_kwargs['clf_only']=False       
# watermarking_order = ['augment','use_PCgrad','sacrifice_kwargs','beta_selection','subgraph_method','reg','perc','clf_epochs','frac','coef_wmk','num_subgraphs','balance_beta_weights','ignore_subgraph_neighbors']
# print("watermarking:", watermarking_order)
# count, [node_classifier, history, subgraph_dict,\
#          all_feature_importances, all_watermark_indices, probas] = dynamic_grid_search(data, dataset_name, debug_multiple_subgraphs, 
#                                                                                        all_dfs, False, variables, watermarking_order,
#                                                                                        count_only=True)
# print(count)


config.augment_kwargs['separate_trainset_from_subgraphs'] = True
config.augment_kwargs['ignore_subgraphs'] = True
perturb=False
perturb_lr = 1e-3
node_classifier, history, subgraph_dict, \
    all_feature_importances, all_watermark_indices, probas = train(data, dataset_name, debug_multiple_subgraphs=False, save=True, print_every=1,perturb=perturb,perturb_lr=perturb_lr)
primary_loss_curve, watermark_loss_curve, final_betas, watermarks, \
    percent_matches, percent_match_mean, percent_match_std, \
        primary_acc_curve, watermark_acc_curve, train_acc, val_acc = get_performance_trends(history, subgraph_dict)
final_plot(history, '', percent_matches, primary_loss_curve, watermark_loss_curve, train_acc)  



NameError: name 'train' is not defined

In [ ]:
data.x[all]

IndexError: only integers, slices (`:`), ellipsis (`...`), None and long or byte Variables are valid indices (got builtin_function_or_method)

In [ ]:
folder = '/Users/janedowner/Desktop/Desktop/IDEAL/Project_2/training_results/computers/archGCN_elu_nLayers3_hDim256_drop0_skipTrue/_3%UnimportantIndices_average_10ClfEpochs_random_fraction0.01_numSubgraphs10_eps0.1_raw_beta_nodeMixUp40_lr0.002_epochs80_coefWmk200_regressionLambda0.1'
probas = pickle.load(open(f'{folder}/probas','rb'))
subgraph_dict = pickle.load(open(f'{folder}/subgraph_dict','rb'))
all_watermark_indices = pickle.load(open(f'{folder}/all_watermark_indices','rb'))
node_classifier = pickle.load(open(f'{folder}/node_classifier','rb'))
history = pickle.load(open(f'{folder}/history','rb'))


In [ ]:
data

NameError: name 'data' is not defined

In [ ]:
import nlopt
import numpy as np

# Define your objective function
def objective(x, grad):
    if grad.size > 0:
        # Compute gradient if needed
        x_tensor = torch.tensor(x, dtype=torch.float32).reshape(original_shape)
        x_tensor.requires_grad = True
        log_logits_sub = node_classifier(x_tensor, edge_index)
        probas_sub = log_logits_sub.clone().exp()

        omit_indices, not_omit_indices = get_omit_indices(x_tensor, this_watermark, ignore_zeros_from_subgraphs=False)
        raw_beta = solve_regression(x_tensor, probas_sub, regression_kwargs['lambda'])
        beta = process_beta(raw_beta, watermark_loss_kwargs['alpha'], omit_indices, watermark_loss_kwargs['scale_beta_method'])
        B_x_W = (beta * this_watermark).clone()
        B_x_W = B_x_W[not_omit_indices]
        balanced_beta_weights = torch.ones_like(B_x_W)
        balanced_beta_weights = balanced_beta_weights[not_omit_indices]
        loss = torch.mean(torch.clamp(watermark_loss_kwargs['epsilon'] - B_x_W, min=0) * balanced_beta_weights)
        loss.backward()

        grad[:] = x_tensor.grad.numpy().flatten()

    return loss.item()

ModuleNotFoundError: No module named 'nlopt'

In [ ]:

from scipy.optimize import minimize
results = []
for sig in subgraph_dict.keys():
    watermark_loss_kwargs = config.watermark_loss_kwargs
    regression_kwargs = config.regression_kwargs
    this_watermark, data_sub, subgraph_node_indices = [subgraph_dict[sig][k] for k in ['watermark','subgraph','nodeIndices']]
    x_sub = data_sub.x
    edge_index = data_sub.edge_index
    original_shape = x_sub.shape
    x_sub_flat = x_sub.flatten()
    # global i
    # i = 0
    def compute_watermark_loss(x_sub_flat):
        # global i
        # i+=1
        # print(/i,end='\r')
        # print('hi')
        x_sub = torch.tensor(x_sub_flat, dtype=torch.float32).reshape(original_shape)
        balanced_beta_weights = torch.ones(x_sub.shape[1])

        log_logits_sub = node_classifier(x_sub, edge_index)
        probas_sub = log_logits_sub.clone().exp()


        omit_indices,not_omit_indices = get_omit_indices(x_sub, this_watermark,ignore_zeros_from_subgraphs=False) #indices where watermark is 0
        raw_beta            = solve_regression(x_sub, probas_sub, regression_kwargs['lambda'])
        beta                = process_beta(raw_beta, watermark_loss_kwargs['alpha'], omit_indices, watermark_loss_kwargs['scale_beta_method'])
        B_x_W = (beta*this_watermark).clone()
        B_x_W = B_x_W[not_omit_indices]
        balanced_beta_weights = balanced_beta_weights[not_omit_indices]
        this_loss_watermark = torch.mean(torch.clamp(watermark_loss_kwargs['epsilon']-B_x_W, min=0)*balanced_beta_weights)
        loss_watermark  = this_loss_watermark
        # print('ok')
        return loss_watermark.item()

    # Ensure maxiter is enforced and debugging output
    options = {'maxfun': 2, 'iprint':110, 'maxls':2, 'maxiter':2}

    # Use the L-BFGS-B algorithm for optimization
    result = minimize(compute_watermark_loss, x_sub_flat, 
                      method='L-BFGS-B', 
                      options=options)

    # The optimal x found by the optimizer
    results.append(result)
    # print(result)


_minimize_lbfgsb options: 2 2
_prepare_scalar_function
C
D
F
init
update_fun: <function ScalarFunction.__init__.<locals>.update_fun at 0x3170d3880>
fun_wrapped
args: ()
FD_METHODS
finite_diff_options: {'method': '2-point', 'rel_step': None, 'abs_step': 1e-08, 'bounds': (array([-inf, -inf, -inf, ..., -inf, -inf, -inf]), array([inf, inf, inf, ..., inf, inf, inf]))}
Help on function update_grad in module scipy.optimize._differentiable_functions:

update_grad()

update_grad: None
not self.g_updated
approx deriv
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_wrapped
args: ()
fun_

KeyboardInterrupt: 

In [ ]:
scipy.optimize

<module 'scipy.optimize' from '/opt/homebrew/Caskroom/miniforge/base/envs/proj_2_env/lib/python3.12/site-packages/scipy/optimize/__init__.py'>

In [ ]:
len(set(torch.wherae(torch.eq(data_copy.x, data.x)==False)[1].tolist()))

767

In [ ]:
data.x[24,0]

tensor(0.)

In [ ]:
data_copy.x[[24,25],[0,1]]

tensor([-4.8543e-05,  0.0000e+00])

In [ ]:
import torch

t = torch.tensor([1,12])
t.numpy()

array([ 1, 12])